In [1]:
import pandas as pd
import numpy as np
import torch

from torch.utils.data import DataLoader

In [2]:
from lstm_scratch import SimpleLSTM
from helpers import CVFConfigForAnalysisDataset

In [ ]:
model_name = "lstm_trained_at_2025_04_10_00_11"

# graph_name = "star_graph_n15"
# graph_name = "star_graph_n7"
# graph_name = "graph_powerlaw_cluster_graph_n7"
graph_name = "graph_random_regular_graph_n7_d4"

In [4]:
device = "cuda" 

In [5]:
# Model class must be defined somewhere
model = torch.load(f"trained_models/{model_name}.pt", weights_only=False)
model.eval()

SimpleLSTM(
  (lstm): GRU(3, 32, batch_first=True)
  (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  (h2o): Linear(in_features=32, out_features=1, bias=True)
)

In [6]:
dataset = CVFConfigForAnalysisDataset(device, graph_name)

Total configs: 27,000.


In [7]:
data = []
# result_df = pd.DataFrame([], columns=['node', 'rank_effect'])

result_df = pd.DataFrame({
    'node': pd.Series(dtype='int'),
    'rank effect': pd.Series(dtype='float')
})

with torch.no_grad():
    test_dataloader = DataLoader(dataset, batch_size=1)

    count = 0
    for batch in test_dataloader:
        for i in range(len(batch[0])):
            frm_idx = batch[1][i].item()
            frm_rank = model(batch[0][i].unsqueeze(0))
            for (
                position,
                to_indx,
            ) in dataset.cvf_analysis.possible_perturbed_state_frm(frm_idx):
                to = dataset[to_indx]
                to_rank = model(to[0].unsqueeze(0))
                rank_effect = (frm_rank - to_rank).item()  # to round off at 0.5
                data.append({'node': position, 'rank effect': rank_effect})

        temp_df = pd.DataFrame(data, columns=['node', 'rank effect'])
        data = []
        result_df = pd.concat([result_df, temp_df], ignore_index=True)


result_df['rank effect'] = np.floor(result_df['rank effect']+0.5)
# result_df.to_csv(f"ml_predictions/{model_name}__{graph_name}__cvf.csv")

print("Done!")


Done!


In [8]:
result_df

,node,rank effect
0,0,-2.0
1,0,-1.0
2,0,-2.0
3,0,-2.0
4,1,-1.0
...,...,...
647995,5,1.0
647996,5,0.0
647997,6,0.0
647998,6,0.0


In [9]:
ml_grp_by_node_re = result_df.groupby(['node', 'rank effect']).size().reset_index(name='ml_count')
ml_grp_by_node_re.to_csv(f"ml_predictions/{model_name}__{graph_name}__cvf_by_node.csv")

In [10]:
ml_grp_by_re = result_df.groupby(['rank effect']).size().reset_index(name='ml_count')
ml_grp_by_re.to_csv(f"ml_predictions/{model_name}__{graph_name}__cvf.csv")

# Full analysis data fetch

In [11]:
import os

results_dir = os.path.join(
    os.getenv("CVF_PROJECT_DIR", ""),
    "cvf-analysis",
    "v2",
    "results",
    "coloring"
)

In [12]:
results_file = f"rank_effects_avg__{graph_name}.csv"
file_path = os.path.join(results_dir, results_file)
f_grp_by_re = pd.read_csv(file_path)
f_grp_by_re = f_grp_by_re.drop(f_grp_by_re.columns[0], axis=1)
f_grp_by_re.rename(columns={'count': 'fa_count'}, inplace=True)

f_grp_by_re

,rank effect,fa_count
0,-3,3828
1,-2,25302
2,-1,169338
3,0,251064
4,1,169338
5,2,25302
6,3,3828


In [13]:
df_grp_by_re = pd.merge(f_grp_by_re, ml_grp_by_re, on='rank effect', how='outer').fillna(0)
df_grp_by_re

,rank effect,fa_count,ml_count
0,-4.0,0.0,29
1,-3.0,3828.0,4124
2,-2.0,25302.0,28810
3,-1.0,169338.0,166393
4,0.0,251064.0,249288
5,1.0,169338.0,166393
6,2.0,25302.0,28810
7,3.0,3828.0,4124
8,4.0,0.0,29


In [14]:
df_grp_by_re.to_csv(f"ml_predictions/{model_name}__{graph_name}__cvf.csv")

In [15]:
results_file = f"rank_effects_by_node_avg__{graph_name}.csv"

file_path = os.path.join(results_dir, results_file)
f_grp_by_node_re = pd.read_csv(file_path)

f_grp_by_node_re = f_grp_by_node_re.melt(id_vars='node', value_vars=set(f_grp_by_node_re.columns)-{'node'}, var_name='rank effect', value_name='fa_count')
f_grp_by_node_re['rank effect'] = f_grp_by_node_re['rank effect'].astype(float)
f_grp_by_node_re

,node,rank effect,fa_count
0,0,3.0,0
1,1,3.0,0
2,2,3.0,0
3,3,3.0,1680
4,4,3.0,2148
5,5,3.0,0
6,6,3.0,0
7,0,-2.0,2871
8,1,-2.0,768
9,2,-2.0,768


In [16]:
df_grp_by_node_re = pd.merge(f_grp_by_node_re, ml_grp_by_node_re, on=['node', 'rank effect'], how='outer').fillna(0)
df_grp_by_node_re

,node,rank effect,fa_count,ml_count
0,0,-3.0,0.0,28
1,0,-2.0,2871.0,3641
2,0,-1.0,29586.0,28917
3,0,0.0,43086.0,42828
4,0,1.0,29586.0,28917
5,0,2.0,2871.0,3641
6,0,3.0,0.0,28
7,1,-3.0,0.0,24
8,1,-2.0,768.0,1044
9,1,-1.0,13632.0,13352


In [17]:
df_grp_by_node_re.to_csv(f"ml_predictions/{model_name}__{graph_name}__cvf_by_node.csv")